In [ ]:
import numpy as np
import pandas as pd
import gc

train = pd.read_csv("train_bool.csv",skipinitialspace=True,index_col='ncodpers',dtype=np.int32)
train.info(verbose=True)

In [ ]:
first_cols=[col for col in train.columns if '2015-01-28' in col]
temp = train.loc[:,first_cols]
train = train.diff(axis=1)
train.loc[:,first_cols]=temp
train[train < 0] = 0
train=train.astype(bool)
del temp
gc.collect()
train.head(50)

In [ ]:
t_not=(train==0).all(axis=1)
train=train[~t_not]
del t_not
gc.collect()
train.head(50)

In [ ]:
#train.head().apply(lambda x: list(x[x==True].index), axis=1)
cols_without_date=[i[:-11] for i in train.columns]
train=train.astype(str)
train.replace('True', pd.Series(cols_without_date,train.columns),inplace=True)
train.replace('False', '',inplace=True)
train.columns=[i[-10:] for i in train.columns]
train.head(50)

In [ ]:
from multiprocessing import Pool
from contextlib import closing

_func = None

def worker_init(func):
  global _func
  _func = func
  

def worker(x):
  return _func(x)


def xmap(func, iterable, processes=None):
   with closing( Pool(processes, initializer=worker_init, initargs=(func,)) ) as p:
    return p.map(worker, iterable)

"""
print(xmap(lambda x: ([i for i in pd.Series(data=x[1],index=x[1].index).dropna().unique()] or [None])[0],
           train.iloc[-30:-1,:]['ult_fec_cli_1t'].iterrows()))
"""
for col in train.columns.unique():
    print(col)
    temp=xmap(lambda x: ' '.join(x[x!='']),train[col].iterrows())
    train=train.drop(columns=col)
    train[col]=temp


In [ ]:
train.reset_index(inplace=True)
train=train.melt(id_vars=['ncodpers'], value_vars=train.columns[1:])
train=train[train['value'].str.strip() != '']
train.head()

In [ ]:
sequence=pd.Series(list(train['ncodpers'].sort_values().unique()))
sequence=pd.Series(sequence.index,sequence)
event=pd.Series(list(train['variable'].sort_values().unique()))
event=pd.Series(event.index,event)
event

In [ ]:
train['ncodpers']=train['ncodpers'].map(sequence)
train['variable']=train['variable'].map(event)

In [ ]:
train['value']=train['value'].apply(lambda x: ' '.join(x.split()))
train.head(20)

In [ ]:
tmp=train['value'].apply(lambda x: x.count("_ult1"))
train.insert(2,'length',tmp)
train

In [ ]:
import csv
train.to_csv('sequencerules_training.txt',sep=' ',index=False, header=False,
             mode='w',quoting=csv.QUOTE_NONE,escapechar='\t')